In [ ]:
# Step 1: Import the necessary libraries
import pandas as pd
import numpy as np
import statsmodels.api as sm
from pandas_datareader import data as pdr

In [ ]:
# Step 2: Load your risk premia strategies data
# Assuming you have a CSV file for each strategy named 'strategy1.csv', 'strategy2.csv', etc.
strategy_files = ['strategy1.csv', 'strategy2.csv', 'strategy3.csv', 'strategy4.csv']
strategies = [pd.read_csv(file, index_col='Date', parse_dates=True) for file in strategy_files]

In [ ]:
# Step 3: Retrieve the Fama-French 4-factor data from the Kenneth R. French data library
ff_factors = pdr.DataReader('F-F_Research_Data_Factors_daily', 'famafrench', start='2024-01-01', end='2024-04-30')[0]
momentum_factor = pdr.DataReader('F-F_Momentum_Factor_daily', 'famafrench', start='2024-01-01', end='2024-04-30')[0]

# Combine the Fama-French factors and the momentum factor into a single DataFrame
ff_factors = ff_factors.join(momentum_factor['Mom   '])

# Convert the factors to daily returns as percentages
ff_factors = ff_factors / 100

# Step 4: Run the Fama-French 4-factor regression for each strategy
results = {}
for i, strategy in enumerate(strategies):
    strategy_name = f'Strategy {i+1}'
    strategy = strategy.rename(columns={'Return': strategy_name})
    
    # Merge strategy returns with the Fama-French factors
    data = strategy.join(ff_factors, how='inner')
    
    # Prepare the data for regression
    X = data[['Mkt-RF', 'SMB', 'HML', 'Mom   ']]
    y = data[strategy_name] - data['RF']
    
    # Add a constant to the independent variables
    X = sm.add_constant(X)
    
    # Run the regression
    model = sm.OLS(y, X).fit()
    
    # Store the results
    results[strategy_name] = model.summary()

# Print the results
for strategy_name, result in results.items():
    print(f"\n{strategy_name} Regression Results:\n")
    print(result)
